indicates True when the corresponding values for that column NAN
- model learns to create diff betn null rows and non null

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.impute import MissingIndicator,SimpleImputer

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/refs/heads/main/day38-missing-indicator/train.csv',usecols=['Age','Fare','Survived'])


In [4]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [6]:
si = SimpleImputer()
X_train_trf = si.fit_transform(X_train)
X_test_trf = si.transform(X_test)

In [9]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train_trf, y_train)
y_pred = clf.predict(X_test_trf)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.6145251396648045

In [10]:
mi = MissingIndicator()
mi.fit(X_train)

MissingIndicator()

In [ ]:
mi.features_  # tells which no of columns have missing values

array([0], dtype=int64)

In [13]:
X_train_missing = mi.transform(X_train)
X_test_missing = mi.transform(X_test)

In [15]:
X_train['Age_NA'] = X_train_missing
X_test['Age_NA'] = X_test_missing

In [16]:
X_train

,Age,Fare,Age_NA
30,40.0,27.7208,False
10,4.0,16.7000,False
873,47.0,9.0000,False
182,9.0,31.3875,False
876,20.0,9.8458,False
...,...,...,...
534,30.0,8.6625,False
584,NaN,8.7125,True
493,71.0,49.5042,False
527,NaN,221.7792,True


In [17]:
si = SimpleImputer()

X_train_trf2 = si.fit_transform(X_train)
X_test_trf2 = si.transform(X_test)

In [19]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train_trf2, y_train)
y_pred = clf.predict(X_test_trf2)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6312849162011173

improved accuracy

### Auto way to use indicator without creating new col

In [20]:
si = SimpleImputer(add_indicator=True)

In [21]:
X_train = si.fit_transform(X_train)
X_test = si.transform(X_test)

In [22]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.6312849162011173

# Auto Select parameters for Imputation

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [25]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/refs/heads/main/day38-missing-indicator/train.csv', usecols=['Age', 'Fare', 'Survived', 'Pclass', 'Sex', 'SibSp','Parch', 'Embarked'])

In [27]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [29]:
numeric_features = ['Age', 'Fare'] # Impute(fill null), Standardize
categorical_features = ['Sex', 'Embarked'] # Impute(fill null), One-hot encode

In [30]:
numeric_trf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_trf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [32]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_trf, numeric_features),
        ('cat', categorical_trf, categorical_features)
    ]
)

In [33]:
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [34]:
from sklearn import set_config

set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked'])])),
                ('classifier', LogisticRegression())])

In [41]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__cat__imputer__strategy': ['constant', 'most_frequent'],
    'classifier__C': [0.1, 1.0, 10, 100]
}

grid_search = GridSearchCV(clf, param_grid, cv=10)

In [42]:
grid_search.fit(X_train, y_train)

print("Best params: ", grid_search.best_params_)

Best params:  {'classifier__C': 1.0, 'preprocessor__cat__imputer__strategy': 'constant', 'preprocessor__num__imputer__strategy': 'mean'}


In [43]:
print("Internal CV score: ", grid_search.best_score_)

Internal CV score:  0.7878521126760563


In [46]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values(by='mean_test_score', ascending=False)
cv_results[['param_classifier__C','param_preprocessor__cat__imputer__strategy','param_preprocessor__num__imputer__strategy','mean_test_score']]


,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
4,1.0,constant,mean,0.787852
5,1.0,constant,median,0.787852
6,1.0,most_frequent,mean,0.787852
7,1.0,most_frequent,median,0.787852
8,10.0,constant,mean,0.787852
9,10.0,constant,median,0.787852
10,10.0,most_frequent,mean,0.787852
11,10.0,most_frequent,median,0.787852
12,100.0,constant,mean,0.787852
13,100.0,constant,median,0.787852
